# Assignment 2

# Karunanayaka Y.S (190301H)

Github Repository: https://github.com/YSK-Machine-Vision-EN2550/Assignment_2.git

In [ ]:
import numpy as np
from scipy . optimize import minimize
from scipy import linalg
import matplotlib . pyplot as plt

In [7]:
N = 100
half_n = N // 2
r = 10
s = r /16
t = np . random . uniform (0 , 2*np . pi , half_n )
n = s*np . random . randn ( half_n )
x , y = ( r + n)*np . cos ( t ) , ( r + n)*np . sin ( t )
X_circ = np . hstack ( ( x . reshape ( half_n , 1 ) , y . reshape ( half_n , 1 ) ) )
m, b = -1, 2
x = np . linspace (-12, 12 , half_n )
y = m*x + b + s*np . random . randn ( half_n )
X_line = np . hstack ( ( x . reshape ( half_n , 1 ) , y . reshape ( half_n , 1 ) ) )
X = np . vstack ( ( X_circ , X_line ) )

In [12]:
# calculate the best fit circle
probability = 0.90
outlier_ratio = 0.40
trsh_dist = 1
s = 3
Itterations = int(np.log(1-probability)/np.log(1-(1-outlier_ratio)**s))

In [61]:
def RANSAC(X_circ):
    # get three random points from the random samples
    r_points = [X_circ[np.random.randint(X_circ.shape[0])] for _ in range(3)]
    x, y = np.hsplit(np.array(r_points), 2)
    
    
    f = ((x[0]**2-x[1]**2+y[0]**2-y[1]**2)*(x[0]-x[2])-(x[0]**2-x[2]**2+y[0]**2-y[2]**2)*(x[0]-x[1]))/(2*((y[0]-y[2])*(x[0]-x[1])-(y[0]-y[1])*(x[0]-x[2])))
    g = ((x[0]**2-x[1]**2+y[0]**2-y[1]**2)*(y[0]-y[2])-(x[0]**2-x[2]**2+y[0]**2-y[2]**2)*(y[0]-y[1]))/(2*((y[0]-y[1])*(x[0]-x[2])-(y[0]-y[2])*(x[0]-x[1])))

    a, b = -g, -f
    r = np.sqrt((x[0] - a)**2 + (y[0] - b)**2)
    
    print(a,b,r)
    
RANSAC(X_circ)

[-7.64165689]
[-0.43587947] [-0.56595828] [9.3406685]


In [41]:
p = [(1,1),(2,4),(5,3)]

In [45]:
A, B = (p[1][1] - p[0][1]) / (p[1][0] - p[0][0]), (p[2][1] - p[1][1]) / (p[2][0] - p[1][0])
P, Q, R, S = p[1][1] + p[0][1], p[2][1] + p[1][1], p[1][0] + p[0][0], p[2][0] + p[1][0]


b = (p[0][0] - p[2][0]) * (A*P - B*Q) / (2 * (A - B))

In [46]:
b

-10.399999999999999

In [66]:
a = np.array([[1,2],[3,4],[5,6]])

In [72]:
a, b = np.hsplit(a, 2)
a = a.T[]

ValueError: array split does not result in an equal division

array([[1, 3, 5]])

In [70]:
a[0]

array([1])